In [1]:
pip install pandas translate pylatexenc dl_translate

Note: you may need to restart the kernel to use updated packages.


In [2]:
# попытка что-то поделать
import pandas as pd
from translate import Translator

In [3]:
# занимает много времени, лучше делать один раз
# заранее берём только те столбцы, в которых содержится текст (остальные не рассматриваем)
df = pd.read_json('100_questions.json')[['body_markdown','tags']]
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 100 entries, 0 to 99
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   body_markdown  100 non-null    object
 1   tags           100 non-null    object
dtypes: object(2)
memory usage: 2.3+ KB


In [4]:
import dl_translate as dlt
# **ОЧЕНЬ** долго загружается, если впервые -- дать подумать
# device="auto" для выбора GPU при возможности, у меня с этим проблемы, т.ч. в целом можно оставить дефолт
# дефолтная модель -- не "mbart50", доки: github.com/xhluca/dl-translate
mt = dlt.TranslationModel("mbart50")

C:\Users\kosty\anaconda3\Lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


In [5]:
pip install tensorflow


  Obtaining dependency information for tensorflow from https://files.pythonhosted.org/packages/93/21/9b035a4f823d6aee2917c75415be9a95861ff3d73a0a65e48edbf210cec1/tensorflow-2.15.0-cp311-cp311-win_amd64.whl.metadata
  Using cached tensorflow-2.15.0-cp311-cp311-win_amd64.whl.metadata (3.6 kB)
  Obtaining dependency information for tensorflow-intel==2.15.0 from https://files.pythonhosted.org/packages/4c/48/1a5a15517f18eaa4ff8d598b1c000300b20c1bb0e624539d702117a0c369/tensorflow_intel-2.15.0-cp311-cp311-win_amd64.whl.metadata
  Using cached tensorflow_intel-2.15.0-cp311-cp311-win_amd64.whl.metadata (5.1 kB)
  Obtaining dependency information for absl-py>=1.0.0 from https://files.pythonhosted.org/packages/a2/ad/e0d3c824784ff121c03cc031f944bc7e139a8f1870ffd2845cc2dd76f6c4/absl_py-2.1.0-py3-none-any.whl.metadata
  Using cached absl_py-2.1.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Obtaining dependency information for flatbuffers>=23.5.2

In [6]:
import tensorflow as tf
print(tf.__version__)


2.15.0


In [7]:
import tensorflow as tf

# Проверяем доступность GPU
if tf.test.gpu_device_name():
    print('GPU доступен: {}'.format(tf.test.gpu_device_name()))
else:
    print('GPU не доступен. Проверьте настройки среды выполнения.')

GPU не доступен. Проверьте настройки среды выполнения.


In [8]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # Измените "0" на номер доступного GPU
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"



In [9]:
pip install Transformers nltk

Note: you may need to restart the kernel to use updated packages.


In [10]:
import nltk
nltk.download("punkt")



[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\kosty\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [11]:
import torch
torch.cuda.empty_cache()

In [ ]:
rows = []
for i, b_ms in enumerate(df['body_markdown']):
    if i == 1000:
      break
    row = {}
    row['markdown'] = b_ms
    torch.cuda.empty_cache()
    sentences = nltk.sent_tokenize(b_ms, "english")
    translated_sentences = []
    batch_size = 4  # Уменьшите batch size
    for j in range(0, len(sentences), batch_size):
        batch = sentences[j:j+batch_size]
        translated_batch = mt.translate(batch, source=dlt.lang.ENGLISH, target=dlt.lang.RUSSIAN)
        translated_sentences.extend(translated_batch)
    row['markdown_rus'] = " ".join(translated_sentences)
    torch.cuda.empty_cache()
    rows.append(row)
    print(i+1,".", ((i+1) / 1217) )


1 . 0.0008216926869350862
2 . 0.0016433853738701725
3 . 0.0024650780608052587
4 . 0.003286770747740345
5 . 0.004108463434675432
6 . 0.0049301561216105174
7 . 0.005751848808545604
8 . 0.00657354149548069
9 . 0.007395234182415777
10 . 0.008216926869350863
11 . 0.009038619556285949


In [ ]:
import csv

# Имя файла CSV
csv_file = 'output.csv'

# Открытие файла CSV в режиме записи
with open(csv_file, 'w', newline='', encoding='utf-8') as file:
    # Создание объекта записи CSV
    writer = csv.DictWriter(file, fieldnames=rows[0].keys(), escapechar='\\')

    # Запись заголовков
    writer.writeheader()

    # Запись данных из листа словарей
    writer.writerows(rows)

In [ ]:
df = pd.read_csv('/content/output.csv')
df.head(10)